In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = "input/"

wine = pd.read_csv(path + 'train.csv')
wine_test = pd.read_csv(path + 'test.csv')

In [5]:
wine.isnull().sum()

fixed acidity             0
volatile acidity          0
citric acid               0
residual sugar            0
chlorides                 0
free sulfur dioxide       0
total sulfur dioxide      0
density                 122
pH                      137
sulphates                 0
alcohol                   0
quality                   0
dtype: int64

In [6]:
#平均値埋め
wine['density'].fillna(wine['density'].mean(), inplace=True)
wine_test['density'].fillna(wine['density'].mean(), inplace=True)

wine['pH'].fillna(wine['pH'].mean(), inplace=True)
wine_test['pH'].fillna(wine['pH'].mean(), inplace=True)

wine.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [7]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1280.00000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000
mean,8.34125,0.527187,0.271383,2.549766,0.087614,15.599844,45.652812,0.997019,3.309405,0.658953,10.427813,5.614297
std,1.74115,0.181153,0.194353,1.452651,0.049233,10.482150,32.914044,0.009760,0.149534,0.175200,1.065322,0.987628
min,4.70000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.951600,2.720000,0.330000,8.400000,2.500000
25%,7.10000,0.390000,0.100000,1.900000,0.070000,7.000000,21.675000,0.991000,3.220000,0.550000,9.500000,4.900000
50%,8.00000,0.520000,0.260000,2.200000,0.079000,13.100000,36.800000,0.997019,3.309405,0.620000,10.200000,5.500000
75%,9.30000,0.640000,0.430000,2.600000,0.090250,20.925000,60.350000,1.002600,3.390000,0.730000,11.100000,6.300000
max,15.90000,1.610000,1.000000,15.500000,0.615000,71.800000,290.300000,1.029000,3.970000,2.050000,15.000000,9.700000


モデル構築

In [41]:
y = wine.quality
X = wine.drop('quality',axis = 1)

X_test = wine_test.iloc[:, 0:].values

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.3, random_state = 42)

In [45]:
#scaling

scaler = preprocessing.StandardScaler().fit(train_x)
train_x_scaled = scaler.transform(train_x)
X_test_scaled = scaler.transform(X_test)

In [15]:
test_x_scaled = scaler.transform(test_x)

In [16]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

/Users/lili/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [17]:
models = []
models.append(('DecisionTree', DecisionTreeRegressor()))
models.append(('RandomForest', RandomForestRegressor()))
models.append(('GradienBoost', GradientBoostingRegressor()))
models.append(('SVR', SVR()))
names = []

In [18]:
for name,model in models:
    kfold = model_selection.KFold(n_splits=5,random_state=42)
    cv_results = model_selection.cross_val_score(model,train_x_scaled,train_y, cv= kfold, scoring = 'neg_mean_absolute_error')
    names.append(name)
    msg  = "%s: %f" % (name, -1*(cv_results).mean())
    print(msg)

DecisionTree: 0.935906
RandomForest: 0.704478
GradienBoost: 0.681625
SVR: 0.680730


In [19]:
model = SVR()
model.fit(train_x_scaled,train_y)
pred_y = model.predict(test_x_scaled)

In [21]:
mean_absolute_error(pred_y,test_y)

0.6879232452309095

In [22]:
SVR?

Init signature: SVR(kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
Docstring:     
Epsilon-Support Vector Regression.

The free parameters in the model are C and epsilon.

The implementation is based on libsvm.

Read more in the :ref:`User Guide <svm_regression>`.

Parameters
----------
C : float, optional (default=1.0)
    Penalty parameter C of the error term.

epsilon : float, optional (default=0.1)
     Epsilon in the epsilon-SVR model. It specifies the epsilon-tube
     within which no penalty is associated in the training loss function
     with points predicted within a distance epsilon from the actual
     value.

kernel : string, optional (default='rbf')
     Specifies the kernel type to be used in the algorithm.
     It must be one of 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed' or
     a callable.
     If none is given, 'rbf' will be used. If a callable is given it is
     used 

In [36]:
def objective(trial):
    # C
    svr_c = trial.suggest_loguniform('svr_c', 1e0, 1e2)
    # epsilon
    epsilon = trial.suggest_loguniform('epsilon', 1e-1, 1e1)
    # SVR
    svr = SVR(C=svr_c, epsilon=epsilon)
    svr.fit(train_x, train_y)
    # 予測
    y_pred = svr.predict(test_x)
    # CrossvalidationのMSEで比較（最大化がまだサポートされていない）
    return mean_squared_error(test_y, y_pred)

In [37]:
import optuna
from sklearn.metrics import mean_squared_error

In [38]:
# optuna
study = optuna.create_study()
study.optimize(objective, n_trials=100)

# 最適解
print(study.best_params)
print(study.best_value)
print(study.best_trial)

[I 2020-01-23 16:28:46,834] Finished trial#0 resulted in value: 0.9358520924428387. Current best value is 0.9358520924428387 with parameters: {'svr_c': 1.6938034781101228, 'epsilon': 1.9323310652638668}.
[I 2020-01-23 16:28:47,003] Finished trial#1 resulted in value: 1.0004085068013115. Current best value is 0.9358520924428387 with parameters: {'svr_c': 1.6938034781101228, 'epsilon': 1.9323310652638668}.
[I 2020-01-23 16:28:47,111] Finished trial#2 resulted in value: 0.903864799817191. Current best value is 0.903864799817191 with parameters: {'svr_c': 3.8854386055389276, 'epsilon': 0.8391522709433139}.
[I 2020-01-23 16:28:47,370] Finished trial#3 resulted in value: 1.0647764919549418. Current best value is 0.903864799817191 with parameters: {'svr_c': 3.8854386055389276, 'epsilon': 0.8391522709433139}.
[I 2020-01-23 16:28:47,452] Finished trial#4 resulted in value: 0.9002484866970102. Current best value is 0.9002484866970102 with parameters: {'svr_c': 41.16769406334698, 'epsilon': 1.483

{'svr_c': 1.0003295622167319, 'epsilon': 0.26741161895009796}
0.8556013249982288
FrozenTrial(number=93, value=0.8556013249982288, datetime_start=datetime.datetime(2020, 1, 23, 16, 29, 1, 682867), datetime_complete=datetime.datetime(2020, 1, 23, 16, 29, 1, 822176), params={'svr_c': 1.0003295622167319, 'epsilon': 0.26741161895009796}, distributions={'svr_c': LogUniformDistribution(high=100.0, low=1.0), 'epsilon': LogUniformDistribution(high=10.0, low=0.1)}, user_attrs={}, system_attrs={'_number': 93}, intermediate_values={}, trial_id=93, state=TrialState.COMPLETE)


In [39]:
model = SVR(C=1.0003295622167319, epsilon=0.26741161895009796)
model.fit(train_x_scaled,train_y)
pred_y = model.predict(test_x_scaled)

mean_absolute_error(pred_y,test_y)

0.6902940734444688

In [48]:
sub = model.predict(X_test_scaled)
submission = pd.read_csv("model/submission.csv")
submission['quality'] = sub